# Cuaderno 18: Flujo multiproducto (multicommodity flow)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un conjunto $K$ de productos; 
* un vector de capacidades $u \in \ZZ^{A}_+$  asociadas a los arcos de $D$; 
* un vector $c \in \ZZ^{A \times K}$ de costos de transporte unitarios asociados a los arcos de $D$ y a los productos de $K$; y, 
* un vector $b \in \ZZ^{V \times K}$ de demandas asociadas a los nodos de $D$ y a los productos de $K$.

El *problema de flujo multiproducto de costo mínimo* consiste en encontrar un flujo $x \in \RR^{A \times K}$  que satisfaga las siguientes condiciones:

* para cada producto $k \in K$ y cada nodo $i \in V$, el flujo neto de $k$ en $i$ (es decir, el flujo total de $k$ sobre los arcos que entran a $i$ menos el flujo total de $k$ sobre los arcos qe salen de $i$) debe ser igual a la demanda $b_i^k$ de $k$ en $i$;  
* para cada arco $(i,j) \in A$, el flujo agregado de todos los productos sobre $(i,j)$ debe ser menor o igual a su capacidad $u_{ij}$;
* el costo total del flujo debe ser mínimo. Este costo se calcula al multiplicar el flujo de cada producto $k \in K$ sobre cada arco $(i,j) \in A$ por su correspondiente costo unitario de transportación $c_{ij}^k$, y sumar estos valores sobre todos los arcos de la red y sobre todos los productos.

Utilizando variables continuas no negativas $x_{ij}^k$ para representar el flujo del producto $k \in K$ sobre el arco $(i, j) \in A$, el problema de flujo multiproducto de costo mínimo puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{k \in K} \sum_{(i,j) \in A} c_{ij}^k x_{ij}^k\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji}^k - \sum_{(i, j) \in A} x_{ij}^k = b_i^k, \quad \forall i \in V, k \in K\\
&\sum_{k \in K} x_{ij}^k \leq u_{ij}, \quad \forall (i, j) \in A, \\
& x_{ij}^k \geq 0, \quad \forall (i, j) \in A, k \in K.
\end{align*}


Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo. Nuestro ejemplo está basado en la instancia [netflow.py](https://www.gurobi.com/documentation/9.0/examples/netflow_py.html) provista como parte de la documentación de Gurobi.

In [1]:
from gurobipy import *

# Productos y nodos del grafo
commodities = tuplelist(['Pencils', 'Pens'])
nodes = tuplelist(['Detroit', 'Denver', 'Boston', 'New York', 'Seattle'])

# Arcos y sus capacidades
arcs, capacity = multidict({
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 })

# Costos unitarios c_{ij}^k asociados a arcos y productos
cost = tupledict({
  ('Pencils', 'Detroit', 'Boston'):   10,
  ('Pencils', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,
  ('Pencils', 'Denver',  'Boston'):   40,
  ('Pencils', 'Denver',  'New York'): 40,
  ('Pencils', 'Denver',  'Seattle'):  30,
  ('Pens',    'Detroit', 'Boston'):   20,
  ('Pens',    'Detroit', 'New York'): 20,
  ('Pens',    'Detroit', 'Seattle'):  80,
  ('Pens',    'Denver',  'Boston'):   60,
  ('Pens',    'Denver',  'New York'): 70,
  ('Pens',    'Denver',  'Seattle'):  30 })

# Demandas asociadas a nodos y productos
inflow = tupledict({
  ('Pencils', 'Detroit'):  -50,
  ('Pencils', 'Denver'):   -60,
  ('Pencils', 'Boston'):    50,
  ('Pencils', 'New York'):  50,
  ('Pencils', 'Seattle'):   10,
  ('Pens',    'Detroit'):  -60,
  ('Pens',    'Denver'):   -40,
  ('Pens',    'Boston'):    40,
  ('Pens',    'New York'):  30,
  ('Pens',    'Seattle'):   30 })

Creamos ahora el objeto modelo y las variables $x_{ij}^k$ del flujo multiproducto. Observar que las variables están indexadas por los conjuntos de productos y de arcos.

Si no se especifica el argumento `vtype` en la llamada a la función `addVars`, por defecto las variables creadas son continuas y no negativas.

Notar por último que se especifica un argumento `obj` en la llamada a `addVars`. El mismo indica el coeficiente de la variable en la función objetivo, con lo cual ya no es necesario llamar al método `setObjective`; por defecto el problema es de minimización.

In [2]:
# Crear el objeto modelo
m = Model('netflow')

# Crear las variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

Using license file /Users/ltorres/gurobi.lic
Academic license - for non-commercial use only


Definimos las restricciones del modelo:

1. Restricciones de satisfacción de la demanda en cada nodo y para cada producto:

In [3]:
# Restricciones de demenda en los nodos
m.addConstrs(
    (flow.sum(h,'*',j) - flow.sum(h,j,'*')  == inflow[h,j]
    for h in commodities for j in nodes), "node")

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

2. Restricciones de capacidad de los arcos, agregadas para todos los productos:

In [4]:
# Restricciones de capacidades en los arcos
m.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for (i,j) in arcs), "cap")

{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

Finalmente, resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [5]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nFlujos optimos para {}:'.format(h))
        for i,j in arcs:
            if solution[h,i,j] >= 0.1:
                print('{} -> {}: {}'.format(i, j, int(solution[h,i,j])))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0x83306cb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.07s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra one simplex iteration after uncrush
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.12 seconds
Optimal objective  5.500000000e+03

Flujos optimos para Pencils:
Detroit -> Boston: 50
Denver -> New York: 50
Denver -> Seattle: 10

Flujos optimos para Pens:
Detroit -> Boston: 30
Detroit -> New York: 30
Denver -> Boston: 10
Denver -> Seattle: 30


### Visualización

Finalmente, podemos visualizar las demandas de los productos, las capacidades de los arcos y los flujos de cada producto empleando los módulos `NetworkX` y `ipycytoscape`:

In [6]:
import networkx as nx
import ipycytoscape
D = nx.DiGraph()
D.add_nodes_from(nodes)
for i in nodes:
    D.nodes[i]['etiq']= '{} ({}, {})'.format(i, inflow['Pencils', i], inflow['Pens', i])
D.add_edges_from(arcs)
for i,j in arcs:
    D.edges[i,j]['cap_flujo'] = '({},{})/{}'.format(int(solution['Pencils',i,j]), int(solution['Pens',i,j]), capacity[i,j])
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'shape': 'square', 'width': '100px','background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': '#9dbaea', 'font-size': '10px', 'label': 'data(cap_flujo)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': '#9dbaea'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'shape': 's…

## Código completo

Se reproduce a continuación el código completo del modelo anterior, tomado del manual de Gurobi y adaptado a nuestra notación.

In [6]:
#!/usr/bin/env python3.7

# Copyright 2020, Gurobi Optimization, LLC

# Solve a multi-commodity flow problem.  Two products ('Pencils' and 'Pens')
# are produced in 2 cities ('Detroit' and 'Denver') and must be sent to
# warehouses in 3 cities ('Boston', 'New York', and 'Seattle') to
# satisfy demand ('inflow[h,i]').
#
# Flows on the transportation network must respect arc capacity constraints
# ('capacity[i,j]'). The objective is to minimize the sum of the arc
# transportation costs ('cost[i,j]').

import gurobipy as gp
from gurobipy import GRB

# Base data
commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

arcs, capacity = gp.multidict({
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'New York'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'New York'): 120,
    ('Denver',  'Seattle'):  120})

# Cost for triplets commodity-source-destination
cost = {
    ('Pencils', 'Detroit', 'Boston'):   10,
    ('Pencils', 'Detroit', 'New York'): 20,
    ('Pencils', 'Detroit', 'Seattle'):  60,
    ('Pencils', 'Denver',  'Boston'):   40,
    ('Pencils', 'Denver',  'New York'): 40,
    ('Pencils', 'Denver',  'Seattle'):  30,
    ('Pens',    'Detroit', 'Boston'):   20,
    ('Pens',    'Detroit', 'New York'): 20,
    ('Pens',    'Detroit', 'Seattle'):  80,
    ('Pens',    'Denver',  'Boston'):   60,
    ('Pens',    'Denver',  'New York'): 70,
    ('Pens',    'Denver',  'Seattle'):  30}

# Demand for pairs of commodity-city
inflow = {
    ('Pencils', 'Detroit'):   50,
    ('Pencils', 'Denver'):    60,
    ('Pencils', 'Boston'):   -50,
    ('Pencils', 'New York'): -50,
    ('Pencils', 'Seattle'):  -10,
    ('Pens',    'Detroit'):   60,
    ('Pens',    'Denver'):    40,
    ('Pens',    'Boston'):   -40,
    ('Pens',    'New York'): -30,
    ('Pens',    'Seattle'):  -30}

# Create optimization model
m = gp.Model('netflow')

# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Arc-capacity constraints
m.addConstrs(
    (flow.sum('*', i, j) <= capacity[i, j] for i, j in arcs), "cap")

# Equivalent version using Python looping
# for i, j in arcs:
#   m.addConstr(sum(flow[h, i, j] for h in commodities) <= capacity[i, j],
#               "cap[%s, %s]" % (i, j))


# Flow-conservation constraints
m.addConstrs(
    (flow.sum(h, '*', j) + inflow[h, j] == flow.sum(h, j, '*')
        for h in commodities for j in nodes), "node")

# Alternate version:
# m.addConstrs(
#   (gp.quicksum(flow[h, i, j] for i, j in arcs.select('*', j)) + inflow[h, j] ==
#     gp.quicksum(flow[h, j, k] for j, k in arcs.select(j, '*'))
#     for h in commodities for j in nodes), "node")

# Compute optimal solution
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h, i, j]))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xc43e5943
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra one simplex iteration after uncrush
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  5.500000000e+03

Optimal flows for Pencils:
Detroit -> Boston: 50
Denver -> New York: 50
Denver -> Seattle: 10

Optimal flows for Pens:
Detroit -> Boston: 30
Detroit -> New York: 30
Denver -> Boston: 10
Denver -> Seattle: 30
